## Masking

### padding mask
Mask all the pad tokens in the batch of sequence. It ensures that the model does not treat padding as the input. The mask indicates where pad value 0 is present: it outputs a 1 at those locations, and a 0 otherwise.

mask 序列中的 pad tokens. 保证模型不会把 pad token 当做输入。mask indicates 在 pad 的位置输出 1，其他位置输出 0

In [1]:
import tensorflow as tf

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    
    # add extra dimensions so that we can add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # [batch_size, 1, 1, seq_len]

这里增加两个 newaxis  主要是为了后面的计算。

In [13]:
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
ans = create_padding_mask(x)
print(ans.shape)
print(ans)

(3, 1, 1, 5)
tf.Tensor(
[[[[0. 0. 1. 1. 0.]]]


 [[[0. 0. 0. 1. 1.]]]


 [[[1. 1. 1. 0. 0.]]]], shape=(3, 1, 1, 5), dtype=float32)


### look-ahead mask

The look-ahead mask is used to mask the future tokens in a sequence. In other words, the mask indicates which entries should not be used.

This means that to predict the third word, only the first and second word will be used. Similarly to predict the fourth word, only the first, second and the third word will be used and so on.

在 decoder 时，预测下一个词的时候需要 mask 序列中之后的词。

In [3]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask   # (seq_len, seq_len)

In [4]:
x = tf.random.uniform((1, 3))
print(x)

tf.Tensor([[0.16677237 0.39485312 0.5738574 ]], shape=(1, 3), dtype=float32)


In [5]:
temp = create_look_ahead_mask(x.shape[1])
temp

<tf.Tensor: id=28, shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 1.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

### tf.linalg.band_part
```python
tf.linalg.band_part(
    input,
    num_lower,
    num_upper,
    name=None
)


band[i, j, k, ..., m, n] = in_band(m, n) * input[i, j, k, ..., m, n]

in_band(m, n) = (num_lower < 0 || (m-n) <= num_lower)) && (num_upper < 0 || (n-m) <= num_upper).
```

input:要输入的张量tensor.

num_lower:下三角矩阵保留的副对角线数量，从主对角线开始计算，相当于下三角的带宽。取值为负数时，则全部保留，矩阵不变。

num_upper:上三角矩阵保留的副对角线数量，从主对角线开始计算，相当于上三角的带宽。取值为负数时，则全部保留，矩阵不变。


In [6]:
input_a = tf.constant([[ 0,  1,  2, 3],
                       [-1,  0,  1, 2],
                       [-2, -1,  0, 1],
                       [-3, -2, -1, 0]], dtype=tf.float32)

In [7]:
tf.linalg.band_part(input_a, 1, -1)  # 下三角保留一个带宽，上三角全部保留

<tf.Tensor: id=33, shape=(4, 4), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.],
       [-1.,  0.,  1.,  2.],
       [ 0., -1.,  0.,  1.],
       [ 0.,  0., -1.,  0.]], dtype=float32)>

In [8]:
tf.linalg.band_part(input_a, 2, 1)  # 下三角保留2个带宽，上三角保留 1 个带宽

<tf.Tensor: id=37, shape=(4, 4), dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.],
       [-2., -1.,  0.,  1.],
       [ 0., -2., -1.,  0.]], dtype=float32)>

In [9]:
# look-ahead mask
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask   # (seq_len, seq_len)

In [10]:
tf.linalg.band_part(tf.ones((5,5)), -1, 0)

<tf.Tensor: id=44, shape=(5, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1.]], dtype=float32)>

In [11]:
look_ahead_mask = create_look_ahead_mask(5)
look_ahead_mask

<tf.Tensor: id=53, shape=(5, 5), dtype=float32, numpy=
array([[0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

## Combined mask

针对 decoder 阶段的第一种attention，也就是 self-attention，既包括 padding mask，也包括 look-ahead mask

In [31]:
tgt = tf.constant([[ 1,2,3,0],
                   [2,3,0,0]], dtype=tf.float32)
tgt

<tf.Tensor: id=200, shape=(2, 4), dtype=float32, numpy=
array([[1., 2., 3., 0.],
       [2., 3., 0., 0.]], dtype=float32)>

In [32]:
look_ahead_mask = create_look_ahead_mask(tf.shape(tgt)[1])   # [tgt_seq_len, tgt_seq_len] 上三角都为1，下三角（包括对角线）都为0
look_ahead_mask

<tf.Tensor: id=221, shape=(4, 4), dtype=float32, numpy=
array([[0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]], dtype=float32)>

In [33]:
dec_targrt_padding_mask = create_padding_mask(tgt)  # [batch, 1, 1, tgt_seq_len]
dec_targrt_padding_mask

<tf.Tensor: id=229, shape=(2, 1, 1, 4), dtype=float32, numpy=
array([[[[0., 0., 0., 1.]]],


       [[[0., 0., 1., 1.]]]], dtype=float32)>

In [34]:
combined_mask = tf.maximum(dec_targrt_padding_mask, look_ahead_mask)
combined_mask

<tf.Tensor: id=231, shape=(2, 1, 4, 4), dtype=float32, numpy=
array([[[[0., 1., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]]],


       [[[0., 1., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 1., 1.],
         [0., 0., 1., 1.]]]], dtype=float32)>

### transformer 中的三种 mask

In [35]:
def create_mask(inp, tgt):
    """ 这里总共要计算三种 masking，
    第一种是 encoding 阶段对于 source 端的 padding  mask
    第二种是 decoder 阶段 self-attention 部分的 masked-multi-head-attention,这里不仅包括 padding mask，还包括 look ahead mask
    第三种是 decoder 阶段 encoder_ouput 和 attented target 的交互 attention,这里仅仅包括 padding mask，而且与第一阶段一致，因为是 encoder output
        作为 key 和 value 值. attented target 是 query.

    :param inp:  [batch, inp_seq_len]
    :param tgt:  [batch, tgt_seq_len]
    :return:
    """
    # 第一种masking: encoding padding mask
    enc_padding_amsk = create_padding_mask(inp)  # [batch, 1, 1, inp_seq_len]

    # 第三种masking: decoding padding mask
    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)  # [batch, 1, 1, inp_seq_len]

    # 第二种masking: combined mask
    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tgt)[1])   # [tgt_seq_len, tgt_seq_len] 上三角都为1，下三角（包括对角线）都为0
    dec_targrt_padding_mask = create_padding_mask(tgt)           # [batch, 1, 1, tgt_seq_len]
    combined_mask = tf.maximum(dec_targrt_padding_mask, look_ahead_mask)    # [batch, 1, tgt_seq_len, tgt_seq_len]
    # 对 [batch, 1,1, tgt_Seq_len] 第三个维度广播之后,然后再进行计算。不仅mask了future信息，还mask对应序列中的padding词
    return enc_padding_amsk, combined_mask, dec_padding_mask